<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/seed7_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pykalman

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.5/248.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [3]:
df = pd.read_csv('/content/de_lds_with_labels(310 seed7).csv')


In [4]:
# Separate features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Encode labels and convert to categorical (7 emotions)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded, num_classes=7)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape: (samples, 62 timesteps, 5 features)
X_reshaped = X_scaled.reshape(-1, 62, 5)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_categorical, test_size=0.2, random_state=42)


In [6]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)


X_train: (55793, 62, 5)
y_train: (55793, 7)
X_test: (13949, 62, 5)
y_test: (13949, 7)


In [7]:
batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)


In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(62, 5)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # 7 emotions
])


In [9]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=test_dataset
)


Epoch 1/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 195s 107ms/step - accuracy: 0.2946 - loss: 1.7514 - val_accuracy: 0.6185 - val_loss: 1.0678
Epoch 2/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 201s 107ms/step - accuracy: 0.6984 - loss: 0.8554 - val_accuracy: 0.8907 - val_loss: 0.3671
Epoch 3/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 189s 108ms/step - accuracy: 0.9090 - loss: 0.2990 - val_accuracy: 0.9568 - val_loss: 0.1613
Epoch 4/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 189s 108ms/step - accuracy: 0.9643 - loss: 0.1290 - val_accuracy: 0.9900 - val_loss: 0.0479
Epoch 5/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 201s 108ms/step - accuracy: 0.9877 - loss: 0.0509 - val_accuracy: 0.9996 - val_loss: 0.0103
Epoch 6/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 187s 107ms/step - accuracy: 0.9792 - loss: 0.0747 - val_accuracy: 1.0000 - val_loss: 0.0028
Epoch 7/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 202s 108ms/step - accuracy: 0.9736 - loss: 0.0958 - val_accuracy: 1.0000 - val_loss: 0.0044
Epoch 8/100
1744/1744 ━━━━━━━━━━━━━━━━━━━━ 196s 112ms/s

KeyboardInterrupt: 

In [11]:
model.save("emotion_model.h5")


In [12]:
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")


NameError: name 'history' is not defined

In [14]:
import pickle # Import the pickle module


In [15]:
with open('eeg_label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)

print("LabelEncoder saved successfully as eeg_label_encoder.pkl")


LabelEncoder saved successfully as eeg_label_encoder.pkl


In [24]:
import numpy as np
import pandas as pd
from scipy.signal import welch
from pykalman import KalmanFilter
from tensorflow.keras.models import load_model
import joblib  # ✅ Use joblib instead of pickle

# EEG Channel List (Updated Order)
channels = [
    "FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8",
    "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8",
    "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8",
    "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8",
    "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8",
    "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8",
    "CB1", "O1", "OZ", "O2", "CB2"
]

sampling_rate = 200
window_size = 1
segment_length = sampling_rate * window_size

freq_bands = {
    "Delta": (1, 3),
    "Theta": (4, 7),
    "Alpha": (8, 13),
    "Beta": (14, 30),
    "Gamma": (31, 50)
}

kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    transition_covariance=0.2 * np.eye(1),
    observation_covariance=0.5 * np.eye(1),
    initial_state_mean=[20],
    initial_state_covariance=5 * np.eye(1)
)

def generate_extreme_positive_eeg():

        t = np.linspace(0, 1, sampling_rate)
        eeg_data = []

        for _ in range(62):
        # Random amplitude and frequency for each wave
          alpha_wave = np.random.uniform(40, 80) * np.sin(2 * np.pi * np.random.uniform(8, 13) * t)
          beta_wave = np.random.uniform(15, 35) * np.sin(2 * np.pi * np.random.uniform(14, 30) * t)
          delta_wave = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(1, 3) * t)
          theta_wave = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(4, 7) * t)
          gamma_wave = np.random.uniform(1, 5) * np.sin(2 * np.pi * np.random.uniform(31, 50) * t)
          noise = np.random.normal(0, np.random.uniform(2, 5), t.shape)

          eeg_channel_signal = delta_wave + theta_wave + alpha_wave + beta_wave + gamma_wave + noise
          eeg_channel_signal = np.clip(eeg_channel_signal, -70, 70)

          eeg_data.append(eeg_channel_signal)

        return np.array(eeg_data)

def extract_de_lds():
    eeg_data = generate_extreme_positive_eeg()
    de_features = []
    for channel_data in eeg_data:
        freqs, psd = welch(channel_data, fs=sampling_rate, nperseg=min(200, segment_length), scaling='density')
        band_de = []
        for band, (low, high) in freq_bands.items():
            band_power = psd[(freqs >= low) & (freqs <= high)]
            band_de_value = 0.5 * np.log10(2 * np.pi * np.e * (np.mean(band_power) + 1e-6))
            band_de.append(band_de_value)
        band_de = np.array(band_de).reshape(-1, 1)
        smoothed_de, _ = kf.filter(band_de)
        de_features.append(smoothed_de.flatten())
    de_features = np.array(de_features)
    de_min, de_max = np.min(de_features), np.max(de_features)
    seed_min, seed_max = 15, 27
    de_lds_values = seed_min + (de_features - de_min) / (de_max - de_min) * (seed_max - seed_min)
    print(f"🟢 Extracted DE_LDS Features - Shape: {de_lds_values.shape}")
    return de_lds_values.flatten()

def predict_emotion():
    sample = extract_de_lds().reshape((1, 62,5))

    # ✅ Load model
    han_model = load_model("/content/emotion_model.h5")

    # ✅ Load y_encoder using joblib
    y_encoder = joblib.load("/content/eeg_label_encoder.pkl")

    prediction = han_model.predict(sample)
    predicted_label = np.argmax(prediction, axis=1)
    decoded_label = y_encoder.inverse_transform(predicted_label)

    print(f"🔵 Predicted Emotion: {decoded_label[0]}")
    return decoded_label[0]

# Run it
predicted_emotion = predict_emotion()

🟢 Extracted DE_LDS Features - Shape: (62, 5)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 690ms/step
🔵 Predicted Emotion: Fear
